In [0]:
import requests
from pyspark.sql.functions import to_date, year, month, dayofmonth
from pyspark.sql.types import StructType, StructField, StringType, DoubleType


def extract_data(data, base="BRL"):
    import requests

    url = f"https://api.apilayer.com/exchangerates_data/{data}"
    headers = {
        "apikey": "JACfv5Rwj1u3Gqmc4bcK8iaB5aC1PilH"
    }
    parameters = {"base": base}

    response = requests.request(
        "GET",
        url,
        headers=headers,
        params=parameters
    )

    if response.status_code != 200:
        raise Exception("Not Found")

    return response.json()


In [0]:
data = extract_data("2024-09-07")

In [0]:
ano, mes, dia = data['date'].split('-')

In [0]:
dbutils.fs.ls("/Volumes/airflow_databricks/default/bronze")

In [0]:
rows = [
    {"moeda": k, "taxa": float(v), "base": data["base"], "data": data["date"]}
    for k, v in data["rates"].items()
]

In [0]:
schema = StructType([
    StructField("moeda", StringType(), True),
    StructField("taxa", DoubleType(), True),
    StructField("base", StringType(), True),
    StructField("data", StringType(), True)
])

df = spark.createDataFrame(rows, schema=schema)

df = df.withColumn("data_dt", to_date("data"))\
      .withColumn("ano",  year("data_dt"))\
      .withColumn("mes",  month("data_dt"))\
      .withColumn("dia",  dayofmonth("data_dt"))\
      .drop("data")

df.show()

In [0]:
df.write \
  .format("delta") \
  .mode("overwrite") \
  .partitionBy("ano","mes","dia")\
  .save("/Volumes/airflow_databricks/default/bronze")

In [0]:
path = "/Volumes/airflow_databricks/default/bronze"

df_all = spark.read.format("delta").load(path)

display(df_all)